## Cardiac MRF Workshop 2022-C

This notebook covers what is being done in `rec_mrf_im` and `match_sig` in `cMRF_Workshop2022_helper`. Here we will load a MRF scan which we have simulated before and go through reconstruction and template matching. 

In [ ]:
# Import required packages
import shutil, os
from pathlib import Path 

import matplotlib.pyplot as plt

import time

import nibabel as nib
import numpy as np

import sirf.Reg as pReg
import sirf.DynamicSimulation as pDS
import sirf.Gadgetron as pMR
from cil.utilities.jupyter import islicer, link_islicer

import auxiliary_functions as aux
import cMRF_Workshop2022_helper as helper

In [ ]:
# Set main paths
# Where is all the data
fpath_base = Path('/mnt/share/SA/')

# Where are we saving (intermediate) results
root_path = Path('/home/sirfuser/devel/Data_mrf_sim/')
root_path.mkdir(exist_ok=True, parents=True)


# These folders will be filled with the formatted input 
# for the simulation and will then also contain the output of the simulation
fpath_in = root_path / "Input"
fpath_in.mkdir(exist_ok=True, parents=True)
fpath_out = root_path / "Output"
fpath_out.mkdir(exist_ok=True, parents=True)

In [ ]:
# Tissue parameters (T1, T2, Rho) : xml
fname_xml = fpath_base / 'XCat/XCAT_TissueParameters_XML.xml'
# 3D tissue segmentation : nifti
fpath_segmentation_nii = fpath_base / 'XCat/label_volume_sa.nii'
# Respiratory motion fields : nifti
fpath_resp_mvf = fpath_base / 'XCat/mvf_resp/'
# Cardiac motion fields : nifti
fpath_card_mvf = fpath_base / 'XCat/mvf_card/'

# MR raw k-space data file as acquisition template : ismrmrd
fname_acquisition_template = fpath_base / 'templates/acquisition_template.h5'

# MRF parameters used for EPG simulation
fname_epg_par = fpath_base / 'Fingerprinting/XCAT_tissue_parameter_list.npz'
# MRF signals describing signal behaviour of tissue types in segmentation simulated with EPG
fname_epg_sig = fpath_base / 'Fingerprinting/XCAT_tissue_parameter_fingerprints.npy'
# MRF dictionary for matching and parameter estimation
fname_dict = fpath_base / 'Fingerprinting/dict_70_1500.npz'

# Prefix for ground truth T1, T2 and rho maps
prefix_ground_truth = fpath_out / 'simulation_gt'


fname_sim = fpath_base / 'Fingerprinting/mrf_simulation.h5'
filenames_parametermaps = 

## A Clean up folders

When we set up the simulation, we need to save several files temporarily. To make sure we don't run into any problems from previous simulations, we go through all the folders and clean them. When you run the following cell you will be asked if you want to delete any old content. Just say 'y' to agree.

In [ ]:
helper.clear_folders([fpath_in, fpath_out])

## B Load the simulated MRF raw data file

In [ ]:
simulated_data = pMR.AcquisitionData(str(fname_sim))

## C Carry out image reconstruction

In [ ]:
# First compute the CSM based on all data
csm = pMR.CoilSensitivityData()
csm.smoothness = 50
csm.calculate(simulated_data)
csm_arr = csm.as_array()

We won't reconstruct one image per radial spoke but set the total number of reconstructed images with the paramter `num_recon_imgs`. The higher this values, the fewer radial lines per image are used and the more accurate the temporal behaviour of the MRF signal is recovered. But of course also the worse the quality of each image.

In [ ]:
# Then activate the time-resolved reconstruction in repetition dimension
num_recon_imgs = 250
simulated_data = aux.activate_timeresolved_reconstruction(simulated_data, num_recon_imgs)

In [ ]:
# Set up a new CSM based on the time-resolved acquisition data
csm = pMR.CoilSensitivityData()

# This step sets up a time-resolved coilmap. The reconstruction checks if for each
# time point a coilmap is present. 
csm.calculate(simulated_data) 

In [ ]:
# But we want to use the coilmap that was computed from the entire dataset
# so we give every repetition the same coilmap
num_reps = csm.as_array().shape[1]
csm_arr = np.tile(csm_arr, (1,num_reps,1,1))

# Unfortunately these two axes have to be swapped.
csm_arr = np.swapaxes(csm_arr, 0, 1)
csm = csm.fill(csm_arr.astype(csm.as_array().dtype))

We will carry out a basic image reconstruction by simply doing gridding of the radial data.

In [ ]:
tstart = time.time()
recon = aux.reconstruct_data(simulated_data, csm)
print(f'Image reconstruction took {time.time()-tstart} seconds.')

The reconstructed images `recon` are an `ImageData` object. In order to get the image information as a numpy array we use the function `as_array()`. 

In [ ]:
recon_result = recon.as_array()
recon_result = np.abs(recon_result)
print(f'The shape of the reconstructed image is {recon_result.shape}')

Then we can visualise it. We will visualse a single image, a view of a line through the image over time (to see the contrast dynamics) and the sum over all dynamic images.

In [ ]:
helper.vis_mrf_im_data(recon_result)

The above reconstruction is simply doing gridding. We can do better by using some iterative algorithms such as iterative SENSE.

In [ ]:
tstart = time.time()
recon = aux.iterative_reconstruct_data(simulated_data, csm)
print(f'Image reconstruction took {time.time()-tstart} seconds.')

Of course it took a bit longer but hopefully the quality is better:

In [ ]:
helper.vis_mrf_im_data(np.abs(recon.as_array()))

## D Dictionary matching

To perform the matching we have to load a pre-computed dictionary. Since we used a sliding window to reconstruct images at lower temporal resolution, but higher image quality the pre-computed dictionary must be brought to the same temporal resolution. This can be done with an auxiliary function does this based on the AcquisitionData we previously used for the images.

In [ ]:
# Load the dictionary
mrfdict = np.load(fname_dict)

dict_theta = mrfdict['dict_theta']
dict_mrf = mrfdict['dict_norm']
dict_mrf = np.transpose( aux.apply_databased_sliding_window(simulated_data, np.transpose(dict_mrf)))

dict_mrf = dict_mrf[0:100,:]
dict_theta = dict_theta[0:100,:]
print("Our dictionary to match is of size {}".format(dict_mrf.shape))

We have to convert the image in the shape `(#time points, #pixels)`

In [ ]:
img_series = recon.as_array()
img_shape = img_series.shape[1:]
img_series_1d = np.transpose(np.reshape(img_series,(img_series.shape[0], -1)))

In [ ]:
# This checks the largest overlap between time-profile and dictionary entries
# if the RAM overflows this will catch it and perform the task in multiple sets.
m0_t1_t2_map_matched = aux.match_dict(dict_mrf, dict_theta, img_series_1d)
m0_t1_t2_map_matched = np.reshape(m0_t1_t2_map_matched, (*img_shape, -1))

## E Visualisation

In [ ]:
helper.vis_m0_t1_t2_mrf_maps([m0_t1_t2_map_matched,])

Now we can also load the ground truth maps to compare

In [ ]:
m0_t1_t2_map_gt = helper.load_gt_maps(filenames_parametermaps)

In [ ]:
helper.vis_m0_t1_t2_mrf_maps([m0_t1_t2_map_gt, m0_t1_t2_map_matched,], method_titles=['Ground truth', 'Motion'])

### Recap
In this notebook we:

- carried out image reconstruction
- estimated T1, T2 and M0 based on a pre-calculated dictionary
- visualised the obtained maps
